In [1]:
import xenaPython as xena

GENES = ['FOXM1', 'TP53']

def get_codes(host, dataset, fields, data):
    "get codes for enumerations"
    codes = xena.field_codes(host, dataset, fields)
    codes_idx = dict([(x['name'], x['code'].split('\t')) for x in codes if x['code'] is not None])
    for i in range(len(fields)):
        if fields[i] in codes_idx:
            data[i] = [None if v == 'NaN' else codes_idx[fields[i]][int(v)] for v in data[i]]
    return data

def get_fields(host, dataset, samples, fields):
    "get field values"
    data = xena.dataset_fetch(host, dataset, samples, fields)
    return data

def get_fields_and_codes(host, dataset, samples, fields):
    "get fields and resolve codes"
    return get_codes( host, dataset, fields, get_fields( host, dataset, samples, fields))

/home/sklave/.local/lib/python3.7/site-packages/xenaPython/__init__.py:110: FutureWarning: Possible nested set at position 7
  re.sub(r"^[^[]+[[]([^]]*)[]].*$", r"\1", query, flags=re.DOTALL))


In [2]:
#
# TCGA target cohort
#

cohort = 'TCGA TARGET GTEx'
host = xena.PUBLIC_HUBS['toilHub']

In [3]:
#dataset: gene expression RNAseq - RSEM tpm
#hub: https://toil.xenahubs.netcohort
#TCGA TARGET GTEx

#dataset IDTcgaTargetGtex_rsem_gene_tpm
#downloadhttps://toil.xenahubs.net/download/TcgaTargetGtex_rsem_gene_tpm.gz; Full metadata
#samples19131
#version2016-09-03
#type of datagene expression RNAseq
#unitlog2(tpm+0.001)
#ID/Gene Mappinghttps://toil.xenahubs.net/download/probeMap/gencode.v23.annotation.gene.probemap; Full metadata
#authorUCSC TOIL RNA-seq recompute
#wranglingData (file names: *.rsem_genes.results) are downloaded, tpm values are extracted, log2(x+0.001) transformed, and combined.
#input data formatROWs (identifiers) x COLUMNs (samples) (i.e. genomicMatrix)




# get samples in cohort
samples = xena.cohort_samples(host, cohort, None)
samples[0: 10]

['TCGA-BR-8590-01',
 'TCGA-P4-A5E8-11',
 'GTEX-QCQG-0326-SM-2I3ES',
 'TCGA-J8-A3YH-06',
 'GTEX-1399Q-2326-SM-5KM2X',
 'GTEX-S32W-0004-SM-2XCE9',
 'TCGA-EJ-7797-11',
 'GTEX-11DXY-0006-SM-5NQ8N',
 'TCGA-HU-8238-11',
 'GTEX-YB5E-0008-SM-4VDT7']

In [4]:
dataset = 'TcgaTargetGtex_rsem_gene_tpm'

expression = xena.dataset_gene_probe_avg(host, dataset, samples, GENES)
expression_by_gene = dict([(g['gene'], g['scores'][0]) for g in expression])
[expression_by_gene.keys(), GENES[0], expression_by_gene[GENES[0]][0:10]]

[dict_keys(['FOXM1', 'TP53']),
 'FOXM1',
 [4.071,
  -0.394,
  -0.7834,
  -0.3022,
  3.379,
  4.686,
  -0.3566,
  -1.086,
  2.473,
  1.541]]

In [5]:
dataset = 'TcgaTargetGTEX_phenotype.txt'
fields = ['_study', '_sample_type']

values = get_fields_and_codes(host, dataset, samples, fields) # list of lists
phenotypes = dict(zip(fields, values)) # index by phenotype
phenotypes['_study'][0:10]

['TCGA',
 'TCGA',
 'GTEX',
 'TCGA',
 'GTEX',
 'GTEX',
 'TCGA',
 'GTEX',
 'TCGA',
 'GTEX']